In [1]:
TRAJECTORY_DIR = '/data/robotarium/trajectories'
MAP_NAME = 'TTIC_ripltown'
EXPERIMENT_NAME = None
ROBOT_NAME = 'autobot01'

In [2]:
# disabling contracts for speed
import contracts
import yaml
import numpy as np
import geometry as geo
import numpy as np
from duckietown_world.world_duckietown.tile_template import load_tile_types
from duckietown_world.geo.measurements_utils import iterate_by_class
from duckietown_world.world_duckietown.tile import get_lane_poses
import duckietown_world as dw
from copy import deepcopy

contracts.disable_all()
m = dw.load_map(MAP_NAME)


INFO:dt-world:duckietown-world 5.0.10
INFO:zuper-typing:zuper-typing 5.0.12
INFO:zuper-commons:zuper-commons 5.0.7
INFO:zuper-typing:In Python 3.6
INFO:dt-world:loading map TTIC_ripltown


In [3]:
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_svg, ipython_draw_html

class Person(dw.PlacedObject):
    def __init__(self, radius, *args, **kwargs):
        self.radius = radius
        dw.PlacedObject.__init__(self, *args, **kwargs)

    def draw_svg(self, drawing, g):
        # drawing is done using the library svgwrite
        c = drawing.circle(center=(0, 0), r=self.radius, fill='pink')
        g.add(c)
        # draws x,y axes
        dw.draw_axes(drawing, g)

    def extent_points(self):
        # set of points describing the boundary 
        L = self.radius
        return [(-L, -L), (+L, +L)]

In [4]:
root = dw.PlacedObject()
import sys, os
import traceback

exp_dir = os.path.join(TRAJECTORY_DIR, EXPERIMENT_NAME) if EXPERIMENT_NAME else TRAJECTORY_DIR
trajectory_file = os.path.join(exp_dir, ROBOT_NAME) + '.yaml'
with open(trajectory_file, 'r') as stream:

    try:
        data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

    timestart = data['begin_time_stamp']
    data_points = len(data['trajectory_data'])
    
    x = np.zeros((data_points,))
    y= np.zeros((data_points,))
    R = np.zeros((3,3, data_points))            
    phi = np.zeros((3, data_points))

    dx = 999.999*np.ones((data_points, ))
    dy = 999.999*np.ones((data_points, ))
    dr = 999.999*np.ones((data_points, ))
    dphi = 999.999*np.ones((data_points, ))

    final_trajectory = []
    for idx, [time, traj] in enumerate(data['trajectory_data'].items()):
        x[idx] = np.array(traj[0])
        y[idx] = np.array(traj[1])
        
        R[:,:,idx] = np.reshape(np.array(traj[3:]), (3,3))
        phi[:,idx] = np.array([np.arctan2(-R[1,2,idx],R[2,2,idx]), 
                               np.arctan2(R[0,2,idx],np.sqrt(R[0,0,idx]**2 + R[0,1,idx]**2)),
                               np.arctan2(-R[0,1,idx], R[0,0,idx])])
        
        z = phi[2,idx]
        points = np.array([x[idx], y[idx]])
        final_trajectory.append([points, z])
    final_array = final_trajectory
    
seqs2 = []
for entry in range(0, len(final_array)):
    x =  (final_array[entry][0][0] )  # -2.2
    y = final_array[entry][0][1] # + 0.8
    alpha = final_array[entry][1]
    q5 = geo.SE2_from_translation_angle([x,y],alpha)
    seqs2.append(q5)
    
timestamps = range(len(seqs2)) # [0, 1, 2, ...]
# SE2Transform is the wrapper for SE2 used by Duckietown World 
transforms = [dw.SE2Transform.from_SE2(_) for _ in seqs2]
seq_me = dw.SampledSequence[object](timestamps, transforms)

In [5]:
counter = 0
center_points2 = []
timestamps2 = []

print('Computing projection...')
for timestamp, pose_object in seq_me:
    counter += 1

    hoi = list(get_lane_poses(m, pose_object.as_SE2()))
    if len(hoi) == 0:
        continue
    else:
        distance_from_left = hoi[0].lane_pose.distance_from_left
        distance_from_right = hoi[0].lane_pose.distance_from_right
        distance_from_center = hoi[0].lane_pose.distance_from_center
        inside = hoi[0].lane_pose.along_lane
        center_point = hoi[0].lane_pose.center_point
        correct_direction = hoi[0].lane_pose.correct_direction
        rel_heading = hoi[0].lane_pose.relative_heading
        if correct_direction is False:
            distance_from_center = distance_from_center 
        if distance_from_center > hoi[0].lane_pose.lateral_left:
            if distance_from_left < distance_from_right:
                print('outside left')
            else:
                print('outside right')
        center_points2.append(hoi[0].lane_pose.center_point)
        timestamps2.append(timestamp)
print('Done!')

Computing projection...
Done!


In [6]:
# we now add a marker for projection in the center point
sequence = dw.SampledSequence[object](timestamps2, center_points2)
#m.set_object("projection2", dw.PlacedObject(), ground_truth=sequence)

m.set_object("me", Person(0.1), ground_truth=seq_me)


In [7]:
ipython_draw_html(m);



INFO:dt-world:area: RectangularArea(pmin=[0. 0.],pmax=[2.92026026 3.51526005])
INFO:dt-world:Written SVG to out/ipython_draw_html/140292454336776/drawing.svg
INFO:dt-world:Written HTML to out/ipython_draw_html/140292454336776/drawing.html
